In [1]:
import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

import json

# = = = = = = = = = = = = = = =

import re
import html as html_lib

In [2]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/input.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：117



,No,OE #,OE
0,1,"Honda: 91501-S1K-003, 91501-STK-003",91501-STK-003
1,2,"Honda: 91501-S1K-003, 91501-STK-003",91501-S1K-003
2,3,Acura: 91503-SZ3-003,91503-SZ3-003
3,4,Honda: 91503-SZ5-003,91503-SZ5-003
4,5,"Honda: 91561-S84-A00-M1, 91561-S84-A21ZB",91561-S84-A00-M1
...,...,...,...
112,113,Honda: 90700-SJ4-000,90700-SJ4-000
113,114,Honda: 75305-SHJ-003,75305-SHJ-003
114,115,Honda: 90602-SFA-004,90602-SFA-004
115,116,Honda: 91514-SAA-003,91514-SAA-003


In [3]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                        1
 OE #    Honda: 91501-S1K-003, 91501-STK-003
 OE                            91501-STK-003
 Name: 0, dtype: object)

In [4]:
import time

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_request = f'''https://www.hondapartsnow.com/genuine/{crawler_series['OE'].replace(' ', '').replace('-', '')}.html'''

        resp = requests.get(url_request,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 404:
            break
        elif resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            with open('./url.html', 'w', encoding='UTF-8') as file:
                file.write(str(soup))

            dict_ = json.loads(html.xpath('//script[@id="initialState"]/text()')[0].split('window.__INITIAL_STORE__ =')[1].replace('undefined', '"undefined"'))
            if dict_:
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

print(soup.prettify())

1
<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=5.0" name="viewport"/>
  <title data-react-helmet="true">
   91501-STK-003 - Genuine Honda Clip, Engine Hood Insulator
  </title>
  <meta content="HondaPartsNow.com offers the great deal for genuine Honda parts, 91501STK003. Clip, Engine Hood Insulator for $0.40. All parts are backed by the Honda's warranty." data-react-helmet="true" name="description"/>
  <link data-react-helmet="true" href="https://www.hondapartsnow.com/genuine/honda~clip~engine~hood~insulator~91501-stk-003.html" rel="canonical"/>
  <link href="/icon_192.png" rel="apple-touch-icon"/>
  <style>
   a,abbr,address,article,aside,audio,b,blockquote,body,canvas,caption,cite,code,dd,del,details,dfn,div,dl,dt,em,fieldset,figcaption,figure,footer,form,h1,h2,h3,h4,h5,h6,header,hgroup,html,i,iframe,img,input,ins,kbd,label,legend,li,mark,menu,nav,object,ol,p,pre,q,samp,sect

In [5]:
resp.status_code == 404

False

In [6]:
h1 = html.xpath('//h1[@class="pn-detail-h1"]')[0]
title = etree.tostring(h1).decode('utf-8')
title = re.sub(r'<.*?>', '', title)
title = re.sub(r' +', ' ', title.strip())
title = html_lib.unescape(title)

title

'Honda 91501-STK-003 Clip, Engine Hood Insulator'

In [7]:
subtitle = html.xpath('//p[@class="pn-detail-sub-desc"]/text()')[0].strip()

subtitle

'2007-2016 Honda 91501STK003'

In [8]:
part_number = dict_['partNumber']['partInfo']['partNumber'].strip()

part_number

'91501-STK-003'

In [9]:
sku = dict_['partNumber']['partInfo']['partNumberAbbr'].strip()

sku

'91501STK003'

In [10]:
type_ = dict_['partNumber']['partInfo']['mainDesc'].strip()

type_

'Clip, Engine Hood Insulator'

In [11]:
description = dict_['partNumber']['partInfo']['subDesc'].strip()

description

'Insulator Clip'

In [12]:
list_specification, dict_count = dict_['partNumber']['partInfo']['specificationList'], {}
for specification in list_specification:
    key = specification['name'].strip()
    if key in dict_count:
        dict_count[key] += 1
    else:
        dict_count[key] = 1

dict_specification, dict_no = {}, {}
for i, specification in enumerate(list_specification):
    key = specification['name'].strip()
    if dict_count[key] == 1:
        dict_specification[str(i)] = {key: specification['desc'].strip()}
    else:
        if key in dict_no:
            dict_no[key] += 1
        else:
            dict_no[key] = 1
        dict_specification[str(i)] = {f'{key}-{dict_no[key]}': specification['desc'].strip()}

dict_specification

{'0': {'Brand': 'Genuine Honda'},
 '1': {'Manufacturer Part Number': '91501-STK-003'},
 '2': {'Part Description': 'Insulator Clip'},
 '3': {'Item Dimensions': '1.3 x 1.3 x 0.5 inches'},
 '4': {'Item Weight': '0.40 Pounds'},
 '5': {'Condition': 'New'},
 '6': {'Fitment Type': 'Direct Replacement'}}

In [13]:
price = dict_['partNumber']['partInfo']['priceInfo']['price'].strip()

price

'0.40'

In [14]:
description_2 = dict_['partNumber']['partInfo']['baseInfoList'][0]['desc'].strip()

description_2

'Honda'

In [15]:
dict_src, list_src = {}, [f'''https://www.hondapartsnow.com{src['largeImg'].strip()}''' for src in dict_['partNumber']['imageList']]
for i, src in enumerate(list_src):
    dict_src[str(i)] = src

dict_src

{'0': 'https://www.hondapartsnow.com/resources/encry/actual-picture/hpn/large/a9a6eacfd86f579199f3cda6b0ff9875/dc6796301e1c04502db339ad18a5b408.jpg',
 '1': 'https://www.hondapartsnow.com/resources/encry/part-picture/hpn/large/5f08abf2870f3000998284167a440ee5/32a198fa1fd33be4296e97142b6b5cf1.png'}

In [16]:
list_head, list_row = [fit['text'].strip() for fit in dict_['partNumber']['fitVehicleTitleList']], dict_['partNumber']['fitVehicleList']

list_head, list_row

(['Year Make Model', 'Body & Trim', 'Emission & Transmission'],
 [['2012 Honda Accord',
   '2 Door EX-L V6, 4 Door EX-L V6, 4 Door EX-V6',
   'KA 5AT, KA 6MT'],
  ['2011 Honda Accord',
   '2 Door EX-L V6, 4 Door EX-L V6, 4 Door EX-V6',
   'KA 5AT, KA 6MT'],
  ['2010 Honda Accord',
   '2 Door EX-L V6, 4 Door EX-L V6, 4 Door EX-V6',
   'KA 5AT, KA 6MT, KL 6MT'],
  ['2009 Honda Accord',
   '2 Door EX, 2 Door EX-L, 2 Door EX-L V6, 2 Door LX-S, 4 Door EX, 4 Door EX (PZEV), 4 Door EX-L, 4 Door EX-L (PZEV), 4 Door EX-L V6, 4 Door EX-V6, 4 Door EX-V6 (PZEV)',
   'KA 5AT, KA 5MT, KA 6MT, KL 5AT, KL 5MT, KL 6MT'],
  ['2008 Honda Accord',
   '2 Door EX, 2 Door EX-L, 2 Door EX-L V6, 2 Door LX, 4 Door EX, 4 Door EX (PZEV), 4 Door EX-L, 4 Door EX-L (PZEV), 4 Door EX-L V6, 4 Door EX-V6, 4 Door EX-V6 (PZEV)',
   'KA 5AT, KA 5MT, KA 6MT, KL 5AT, KL 5MT, KL 6MT'],
  ['2009 Honda CR-V',
   '5 Door EX (2WD), 5 Door EX (4WD), 5 Door EX-L (2WD), 5 Door EX-L (4WD), 5 Door LX (2WD), 5 Door LX (4WD)',
   'KA 5

In [17]:
url = dict_['partNumber']['canonical'].strip()

url

'https://www.hondapartsnow.com/genuine/honda~clip~engine~hood~insulator~91501-stk-003.html'

In [18]:
fit = dict_['partNumber']['fitModel'].strip()

fit

'2007-2016 Honda'

In [19]:
vehicles = '\n'.join(sorted([vehicle['name'].strip() for vehicle in dict_['partNumber']['pnSeo']['vehicles'][0]['links']]))

print(vehicles)

2007-2009 Honda CR-V
2007-2010 Honda Odyssey
2007-2011 Honda Civic
2008-2012 Honda Accord


In [20]:
list_dict_correct_temp = [{'No': crawler_series['No'],
                           'Part Number': part_number,
                           'Type': type_,
                           'Title': title,
                           'Subtitle': subtitle,
                           'SKU': sku,
                           'Price': price,
                           'Description 1': description,
                           'Description 2': description_2,
                           'Fit': fit,
                           'Vehicles': vehicles,
                           'Picture': '',
                           'Url': url,
                           'Json_Src': json.dumps(dict_src),
                           'Json_Specification': json.dumps(dict_specification),
                           'Row': i + 1} for i in range(len(list_row))]

list_dict_correct_temp

[{'No': 1,
  'Part Number': '91501-STK-003',
  'Type': 'Clip, Engine Hood Insulator',
  'Title': 'Honda 91501-STK-003 Clip, Engine Hood Insulator',
  'Subtitle': '2007-2016 Honda 91501STK003',
  'SKU': '91501STK003',
  'Price': '0.40',
  'Description 1': 'Insulator Clip',
  'Description 2': 'Honda',
  'Fit': '2007-2016 Honda',
  'Vehicles': '2007-2009 Honda CR-V\n2007-2010 Honda Odyssey\n2007-2011 Honda Civic\n2008-2012 Honda Accord',
  'Picture': '',
  'Url': 'https://www.hondapartsnow.com/genuine/honda~clip~engine~hood~insulator~91501-stk-003.html',
  'Json_Src': '{"0": "https://www.hondapartsnow.com/resources/encry/actual-picture/hpn/large/a9a6eacfd86f579199f3cda6b0ff9875/dc6796301e1c04502db339ad18a5b408.jpg", "1": "https://www.hondapartsnow.com/resources/encry/part-picture/hpn/large/5f08abf2870f3000998284167a440ee5/32a198fa1fd33be4296e97142b6b5cf1.png"}',
  'Json_Specification': '{"0": {"Brand": "Genuine Honda"}, "1": {"Manufacturer Part Number": "91501-STK-003"}, "2": {"Part Descr

In [21]:
for i, row in enumerate(list_row):
    for j, head in enumerate(list_head):
        list_dict_correct_temp[i][head] = row[j]

list_dict_correct_temp

[{'No': 1,
  'Part Number': '91501-STK-003',
  'Type': 'Clip, Engine Hood Insulator',
  'Title': 'Honda 91501-STK-003 Clip, Engine Hood Insulator',
  'Subtitle': '2007-2016 Honda 91501STK003',
  'SKU': '91501STK003',
  'Price': '0.40',
  'Description 1': 'Insulator Clip',
  'Description 2': 'Honda',
  'Fit': '2007-2016 Honda',
  'Vehicles': '2007-2009 Honda CR-V\n2007-2010 Honda Odyssey\n2007-2011 Honda Civic\n2008-2012 Honda Accord',
  'Picture': '',
  'Url': 'https://www.hondapartsnow.com/genuine/honda~clip~engine~hood~insulator~91501-stk-003.html',
  'Json_Src': '{"0": "https://www.hondapartsnow.com/resources/encry/actual-picture/hpn/large/a9a6eacfd86f579199f3cda6b0ff9875/dc6796301e1c04502db339ad18a5b408.jpg", "1": "https://www.hondapartsnow.com/resources/encry/part-picture/hpn/large/5f08abf2870f3000998284167a440ee5/32a198fa1fd33be4296e97142b6b5cf1.png"}',
  'Json_Specification': '{"0": {"Brand": "Genuine Honda"}, "1": {"Manufacturer Part Number": "91501-STK-003"}, "2": {"Part Descr

In [22]:
list_dict_correct.extend(list_dict_correct_temp)

list_dict_correct

[{'No': 1,
  'Part Number': '91501-STK-003',
  'Type': 'Clip, Engine Hood Insulator',
  'Title': 'Honda 91501-STK-003 Clip, Engine Hood Insulator',
  'Subtitle': '2007-2016 Honda 91501STK003',
  'SKU': '91501STK003',
  'Price': '0.40',
  'Description 1': 'Insulator Clip',
  'Description 2': 'Honda',
  'Fit': '2007-2016 Honda',
  'Vehicles': '2007-2009 Honda CR-V\n2007-2010 Honda Odyssey\n2007-2011 Honda Civic\n2008-2012 Honda Accord',
  'Picture': '',
  'Url': 'https://www.hondapartsnow.com/genuine/honda~clip~engine~hood~insulator~91501-stk-003.html',
  'Json_Src': '{"0": "https://www.hondapartsnow.com/resources/encry/actual-picture/hpn/large/a9a6eacfd86f579199f3cda6b0ff9875/dc6796301e1c04502db339ad18a5b408.jpg", "1": "https://www.hondapartsnow.com/resources/encry/part-picture/hpn/large/5f08abf2870f3000998284167a440ee5/32a198fa1fd33be4296e97142b6b5cf1.png"}',
  'Json_Specification': '{"0": {"Brand": "Genuine Honda"}, "1": {"Manufacturer Part Number": "91501-STK-003"}, "2": {"Part Descr

In [23]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 OE #                         Honda: 91501-S1K-003, 91501-STK-003
 OE                                                 91501-STK-003
 Request_Url    https://www.hondapartsnow.com/genuine/91501STK...
 Name: 0, dtype: object]

In [24]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No', 'Row'],
                                                             ascending=[True, True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_part.xlsx', index=False)

df_correct

,No,Part Number,Type,Title,Subtitle,SKU,Price,Description 1,Description 2,Fit,Vehicles,Picture,Url,Json_Src,Json_Specification,Row,Year Make Model,Body & Trim,Emission & Transmission
0,1,91501-STK-003,"Clip, Engine Hood Insulator","Honda 91501-STK-003 Clip, Engine Hood Insulator",2007-2016 Honda 91501STK003,91501STK003,0.40,Insulator Clip,Honda,2007-2016 Honda,2007-2009 Honda CR-V\n2007-2010 Honda Odyssey\...,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf...",1,2012 Honda Accord,"2 Door EX-L V6, 4 Door EX-L V6, 4 Door EX-V6","KA 5AT, KA 6MT"
1,1,91501-STK-003,"Clip, Engine Hood Insulator","Honda 91501-STK-003 Clip, Engine Hood Insulator",2007-2016 Honda 91501STK003,91501STK003,0.40,Insulator Clip,Honda,2007-2016 Honda,2007-2009 Honda CR-V\n2007-2010 Honda Odyssey\...,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf...",2,2011 Honda Accord,"2 Door EX-L V6, 4 Door EX-L V6, 4 Door EX-V6","KA 5AT, KA 6MT"
2,1,91501-STK-003,"Clip, Engine Hood Insulator","Honda 91501-STK-003 Clip, Engine Hood Insulator",2007-2016 Honda 91501STK003,91501STK003,0.40,Insulator Clip,Honda,2007-2016 Honda,2007-2009 Honda CR-V\n2007-2010 Honda Odyssey\...,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf...",3,2010 Honda Accord,"2 Door EX-L V6, 4 Door EX-L V6, 4 Door EX-V6","KA 5AT, KA 6MT, KL 6MT"
3,1,91501-STK-003,"Clip, Engine Hood Insulator","Honda 91501-STK-003 Clip, Engine Hood Insulator",2007-2016 Honda 91501STK003,91501STK003,0.40,Insulator Clip,Honda,2007-2016 Honda,2007-2009 Honda CR-V\n2007-2010 Honda Odyssey\...,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf...",4,2009 Honda Accord,"2 Door EX, 2 Door EX-L, 2 Door EX-L V6, 2 Door...","KA 5AT, KA 5MT, KA 6MT, KL 5AT, KL 5MT, KL 6MT"
4,1,91501-STK-003,"Clip, Engine Hood Insulator","Honda 91501-STK-003 Clip, Engine Hood Insulator",2007-2016 Honda 91501STK003,91501STK003,0.40,Insulator Clip,Honda,2007-2016 Honda,2007-2009 Honda CR-V\n2007-2010 Honda Odyssey\...,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf...",5,2008 Honda Accord,"2 Door EX, 2 Door EX-L, 2 Door EX-L V6, 2 Door...","KA 5AT, KA 5MT, KA 6MT, KL 5AT, KL 5MT, KL 6MT"
5,1,91501-STK-003,"Clip, Engine Hood Insulator","Honda 91501-STK-003 Clip, Engine Hood Insulator",2007-2016 Honda 91501STK003,91501STK003,0.40,Insulator Clip,Honda,2007-2016 Honda,2007-2009 Honda CR-V\n2007-2010 Honda Odyssey\...,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf...",6,2009 Honda CR-V,"5 Door EX (2WD), 5 Door EX (4WD), 5 Door EX-L ...",KA 5AT
6,1,91501-STK-003,"Clip, Engine Hood Insulator","Honda 91501-STK-003 Clip, Engine Hood Insulator",2007-2016 Honda 91501STK003,91501STK003,0.40,Insulator Clip,Honda,2007-2016 Honda,2007-2009 Honda CR-V\n2007-2010 Honda Odyssey\...,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf...",7,2008 Honda CR-V,"5 Door EX (2WD), 5 Door EX (4WD), 5 Door EX-L ...",KA 5AT
7,1,91501-STK-003,"Clip, Engine Hood Insulator","Honda 91501-STK-003 Clip, Engine Hood Insulator",2007-2016 Honda 91501STK003,91501STK003,0.40,Insulator Clip,Honda,2007-2016 Honda,2007-2009 Honda CR-V\n2007-2010 Honda Odyssey\...,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1""

In [25]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_part_error.xlsx', index=False)

df_error

,No,OE #,OE,Request_Url
0,1,"Honda: 91501-S1K-003, 91501-STK-003",91501-STK-003,https://www.hondapartsnow.com/genuine/91501STK...
